In [2]:
# initial imports

import h5py
import numpy as np
import pickle
from PIL import Image
from npy_append_array import NpyAppendArray
import random

import torchvision
from torchvision import transforms
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights, resnet34, resnet18
import torch.optim as optim

'''
from scipy.stats import zscore
import matplotlib as mpl
from matplotlib import pyplot as plt
'''

'\nfrom scipy.stats import zscore\nimport matplotlib as mpl\nfrom matplotlib import pyplot as plt\n'

In [1]:
# Set session
year = '2026'
month = '11'
day = '30'

type = 'model'
dataset = 'Nat305'

In [3]:
with open(f'./{year}_{month}_{day}/natimg_path_list_{year}{month}{day}', 'rb') as fp:   # Unpickling
    natimg_path_list = pickle.load(fp)

unique_img_paths = list(set(natimg_path_list))
len(unique_img_paths)

305

In [4]:
# Initialize model

if type == 'rand':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
elif type == 'model':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=ResNet50_Weights.DEFAULT)
elif type == 'barlow':
    model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
elif type == 'model34':
    model = torchvision.models.resnet34(pretrained=True)
elif type == 'model18':
    model = torchvision.models.resnet18(pretrained=True)

model.eval()

Using cache found in /Users/parsatalaie/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# def hook

def hook(module, input, output):
    conv_outputs[i] = output[0].flatten().unsqueeze(0)

handle = model.layer4[0].conv2.register_forward_hook(hook)
#handle = model.avgpool.register_forward_hook(hook)

layer = 'conv4'
sublayer = '02'
size = 25088 #25088 for 402, 2048 for avgpool

In [6]:
# fix image, use model
model_outputs = torch.zeros(1, 1000)
conv_outputs = torch.zeros((len(unique_img_paths), size))

for i in range(len(unique_img_paths)):
    input_image = Image.open(unique_img_paths[i]).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    with torch.no_grad():
        output = model(input_batch)
    if layer == 'final':
        model_outputs = torch.cat((model_outputs, output[0].unsqueeze(0)), dim=0)

if layer == 'final':
    outputs_np = model_outputs.detach().cpu().numpy()
else:
    outputs_np = conv_outputs.detach().cpu().numpy()

outputs_np.shape

(305, 25088)

In [7]:
unique_img_dict = {path:outputs_np[i] for i, path in enumerate(unique_img_paths)}

In [8]:
final_outputs_np = np.zeros((len(natimg_path_list), size))

In [9]:
final_outputs_np.shape

(2970, 25088)

In [10]:
for i in range(len(natimg_path_list)):
    final_outputs_np[i] = unique_img_dict[natimg_path_list[i]]

In [11]:
# Old save (not with dictionary/dataset)

In [12]:
np.save(f'./{year}_{month}_{day}/marm_{type}_{layer}{sublayer}outputs_{year}{month}{day}', final_outputs_np)

In [7]:
# INITIAL CREATION

np.save(f'./Model Outputs/marm_{type}_{layer}{sublayer}_outputs_{dataset}', final_outputs_np)

In [3]:
# Initial model weight save

torch.save(model.state_dict(), 'rand_weights.pth')

In [53]:
# Get trial counts for each image in train list

img_count = len(set(natimg_path_list))

count_arr = np.zeros((natimg_path_list.count(max(natimg_path_list)), img_count))
count_arr[:] = np.nan

for i in range(img_count):
    path = f'/Users/parsatalaie/Downloads/rust_natimgs/Nat300_{i+1}.png'
    idx_list = [i for i, j in enumerate(natimg_path_list) if j == path]
    for x, idx in enumerate(idx_list):
        count_arr[x, i] = idx

In [55]:
# Save count_arr

np.save(f'./{year}_{month}_{day}/img_idx_{year}{month}{day}', count_arr)

In [ ]:
# OLD MODEL LOOP
model_outputs = torch.zeros(1, 1000)

for i in range(3000, 3364):
    img = Image.open(natimg_path_list[i]).convert('RGB')
    img_np = np.transpose(np.asarray(img))
    img_tens = torch.from_numpy(img_np.astype('float32')).unsqueeze(0)
    img_transformed = preprocess(img_tens)
    output = model(img_transformed)
    model_outputs = torch.cat((model_outputs, output), dim=0)

outputs_np = model_outputs.detach().cpu().numpy()
outputs_np = np.delete(outputs_np, 0, 0)

outputs_np.shape